In [1]:
import sys
import os
ROOT_DIR = os.path.abspath("/MoStress/")
sys.path.append(ROOT_DIR)

In [2]:
import json
from os.path import join

import numpy as np
import pandas as pd
from kerasbeats import NBeatsModel, prep_time_series
from utils.preprocessingCheckpoint import getPreprocessingCheckpoint


In [3]:
trainData, validationData = getPreprocessingCheckpoint()

In [4]:
dataset = {
    "features": trainData["features"],
    "targets": trainData["targets"],
    "weights": trainData["weights"],
}

In [5]:
inputTest = dataset["features"][0]

inputTest

array([[-0.04906158,  0.32236075,  2.31629804, -0.70362568,  0.35170601],
       [ 0.00304426, -0.309945  ,  2.28104558, -0.45453641,  0.43153586],
       [ 0.62631692,  0.45954869,  2.19640463, -0.31583771,  0.497078  ],
       ...,
       [-0.00608964, -0.49451708,  1.34191815, -2.03553343, -0.74628732],
       [ 0.23747035,  0.34576267,  1.38512626, -1.23622251, -0.75530661],
       [-2.68725794,  1.26433206,  1.45325614, -2.09487152, -0.75049123]])

In [23]:
from tensorflow import keras, convert_to_tensor, stack
from random import randint, seed
seed(42)

In [7]:
timeInputLayers = []
nbeatsLayers = []
for i in range(inputTest.shape[1]):
    time_input = keras.layers.Input(shape = (7, ))
    nbeats = NBeatsModel(model_type = 'generic', lookback = 7, horizon = 1).build_layer()
    timeInputLayers.append(time_input)
    nbeatsLayers.append(nbeats.model_layer(time_input))

2022-09-22 18:26:44.822160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
concat = keras.layers.Concatenate()(nbeatsLayers)
output = keras.layers.Dense(3, activation="softmax")(concat)
model = keras.Model(inputs = timeInputLayers, outputs = output)

In [9]:
model.compile(loss = 'sparse_categorical_crossentropy')

In [28]:
x = [ convert_to_tensor(inputTest[:, i]) for i in range(inputTest.shape[1]) ]
y = [
    np.array(
        [
            np.int64(randint(0, 2))
            for _ in range(inputTest.shape[0])
        ]
    )
    for _ in range(inputTest.shape[1])
]

In [32]:
x = stack(x)
y = convert_to_tensor(stack(y))

In [30]:
model.fit(x, y, verbose = True)

ValueError: in user code:

    File "/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/keras/utils/traceback_utils.py",